In [ ]:
!pip install sentencepiece
!pip install transformers

In [ ]:
import torch
import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("raquiba/sarcasm-detection-BanglaSARC")
model = AutoModelForSequenceClassification.from_pretrained("raquiba/sarcasm-detection-BanglaSARC")
model.eval()
model.to(device)

tokenizer_config.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/845 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/Sarcasm_Ner_Converted_Dataset.csv')
df

,original sentence,ner converted,Name,original lebel
0,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,আফা মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,[],1
1,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,বোন মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,[],1
2,ভাই মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে পো...,ভাবী মিথ্যা কথা বইলেন না আপনি ভিপিএন ইউজ করে প...,[],1
3,নিশ্চয় মিমি সাংবাদিককে কোনো কিছু দেওয়ার প্রতিশ...,নিশ্চয় হাবিবা মহিলা সাংবাদিককে কোনো কিছু দেও...,"['মিমি', 'মিমি']",1
4,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসের ...,দুইদিন বৃষ্টিতে একটু সুস্থ হইছে । ভাদ্র মাসে...,[],1
...,...,...,...,...
9840,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো আফা অনেক আগে থেকে ত...,[],0
9841,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো বোন অনেক আগে থেকে ত...,[],0
9842,ডিজিটাল মিটার চালু হইছে তো ভাই অনেক আগে থেকে ত...,ডিজিটাল মিটার চালু হইছে তো ভাবী অনেক আগে থেকে ...,[],0
9843,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার সম্রাট সাক...,আমরা ঢাকাবাসীর গৌরবান্বিত যে টাইগার সম্রাজ্ঞী ...,['সাকিব'],0


In [ ]:
original_sentences = df['original sentence']
converted_sentences = df['ner converted']
original_lebel = df['original lebel']
Name = df['Name']

In [ ]:
original_predictions = []

for txt in original_sentences:
  tokens = tokenizer(str(txt), max_length=512, padding="max_length", truncation=True, return_tensors="pt")
  input_ids = tokens.input_ids
  attention_mask = tokens.attention_mask
  logits = model(input_ids = input_ids.to(device), attention_mask=attention_mask.to(device)).logits
  predicted_class_id = logits.argmax().item()
  original_predictions.append(predicted_class_id)

In [ ]:
converted_predictions = []

for txt in converted_sentences:
  tokens = tokenizer(str(txt), max_length=512, padding="max_length", truncation=True, return_tensors="pt")
  input_ids = tokens.input_ids
  attention_mask = tokens.attention_mask
  logits = model(input_ids = input_ids.to(device), attention_mask=attention_mask.to(device)).logits
  predicted_class_id = logits.argmax().item()
  converted_predictions.append(predicted_class_id)

In [ ]:
df['original predictions'] = original_predictions
df['converted predictions'] = converted_predictions

In [ ]:
# Compare predicted labels and generate result
comparison_results = []
for i in range(len(original_predictions)):
    if original_predictions[i] == converted_predictions[i]:
        comparison_results.append(0)  # Labels match
    else:
        comparison_results.append(1)  # Labels don't match

# Print predicted value and biasness
print("Predicted Value for Original Sentence\tPredicted Value for ner converted\tPredict Biasness")
for orig_label, conv_label, bias in zip(original_predictions, converted_predictions, comparison_results):
    print(f"{orig_label}\t\t\t\t{conv_label}\t\t\t\t{bias}")

Streaming output truncated to the last 5000 lines.
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0				0				0
0

In [ ]:
import pandas as pd

print("Length checks:")
print("Original Sentences:", len(df['original sentence']))
print("Converted Sentences:", len(df['ner converted']))
print("Original Predictions:", len(original_predictions))
print("Converted Predictions:", len(converted_predictions))
print("Comparison Results:", len(comparison_results))

# if all lengths match
if len(df['original sentence']) == len(df['ner converted']) == len(original_predictions) == len(converted_predictions) == len(comparison_results):
    Sentance_data = {
        "Original Sentence": df['original sentence'],
        "Converted Sentence": df['ner converted'],
        "Name" : df['Name'],
        "original_lebel" : df['original lebel'],
        "Predicted Value for Original Sentence": original_predictions,
        "Predicted Value for ner converted": converted_predictions,
        "Predict Biasness": comparison_results
    }

    df_results = pd.DataFrame(Sentance_data)
    csv_location = "/content/drive/MyDrive/Thesis/Thesis Restart/Prediction_Final_Dataset/Sarcasm_Final_&_Prediction_results.csv"
    df_results.to_csv(csv_location, index=False)
    print(f"CSV file saved successfully at: {csv_location}")
else:
    print("Data length mismatch. Please check the data inputs.")


Length checks:
Original Sentences: 9845
Converted Sentences: 9845
Original Predictions: 9845
Converted Predictions: 9845
Comparison Results: 9845
CSV file saved successfully at: /content/drive/MyDrive/Thesis/Thesis Restart/Prediction_Final_Dataset/Sarcasm_Final_&_Prediction_results.csv
